In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'


In [ ]:
%%time
import subprocess

# 셸 명령어 실행
p = subprocess.Popen(['python', 'train.py','--device','0','--workers','4','--weights','yolov7-tiny.pt', '--batch', '16', '--epochs', '300', '--data', 'C:/Users/User/Desktop/cross_data/data.yaml', '--cfg', 'C:/Users/User/Desktop/yujung/yolov7-main/cfg/training/yolov7-tiny.yaml', '--hyp','C:/Users/User/Desktop/yujung/yolov7-main/data/hyp.scratch.tiny.yaml','--name', 'cross_data_7'], stdout=subprocess.PIPE)

# 명령어 실행 동안 로그 출력
while True:
    output = p.stdout.readline().decode()
    if output == '' and p.poll() is not None:
        break
    if output:
        print(output.strip())

In [1]:
import rpyc
import time

RPYC_SERVER_IP = '192.168.137.3'
RPYC_SERVER_PORT = 18812

conn = rpyc.classic.connect(RPYC_SERVER_IP, port=RPYC_SERVER_PORT)

# 모터 포트와 모터 타입 설정
motor_port_a = 'outA'
motor_port_b = 'outB'



# 모터 객체 생성
m = conn.modules['ev3dev2.motor']


motor_a = m.LargeMotor(motor_port_a)
motor_b = m.MediumMotor(motor_port_b)
motor_a.reset()  # 모터 초기화
motor_b.reset()
SortMotor = motor_b

In [2]:
SortMotor.position=0
SortMotor.run_to_abs_pos(position_sp=-87, speed_sp = 550)
SortMotor.wait_until_not_moving()
print("현재 모터의 위치:", SortMotor.position)
SortMotor.run_to_abs_pos(position_sp=-32, speed_sp = 550)

SortMotor.wait_until_not_moving()
print("현재 모터의 위치:", SortMotor.position)

현재 모터의 위치: -118
현재 모터의 위치: -118
현재 모터의 위치: -10
현재 모터의 위치: -10


In [3]:
SortMotor.position=0
SortMotor.run_to_abs_pos(position_sp=86, speed_sp = 550)
SortMotor.wait_until_not_moving()
print("현재 모터의 위치:", SortMotor.position)
SortMotor.run_to_abs_pos(position_sp=35, speed_sp = 550)
SortMotor.wait_until_not_moving()
print("현재 모터의 위치:", SortMotor.position)

현재 모터의 위치: 114
현재 모터의 위치: 114
현재 모터의 위치: 16
현재 모터의 위치: 16


In [1]:
from datetime import datetime
import sys
from PyQt5 import uic
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
from PyQt5.QtCore import *
import os
import pandas as pd
import sys
import test_test
import argparse
import codecs
from openpyxl import Workbook

def resource_path(*relative_Path_AND_File):
    
    try:
        # PyInstaller creates a temp folder and stores path in _MEIPASS
        base_path = getattr(
            sys,
            '_MEIPASS',
            os.path.dirname(os.path.abspath(__file__))
        )
        
    except Exception:
        base_path = os.path.abspath(".")

    return os.path.join(base_path, *relative_Path_AND_File)
form = resource_path("legofinal.ui")
form_class = uic.loadUiType(form)[0]

import rpyc
import time

RPYC_SERVER_IP = '192.168.137.3'
RPYC_SERVER_PORT = 18812

conn = rpyc.classic.connect(RPYC_SERVER_IP, port=RPYC_SERVER_PORT)

# 모터 포트와 모터 타입 설정
motor_port_a = 'outA'
motor_port_b = 'outB'
#motor_port_c = 'outC'
#motor_port_d = 'outD'
#color_sensor_port = 'in1'



# 모터 객체 생성
m = conn.modules['ev3dev2.motor']
s = conn.modules['ev3dev2.sensor.lego']


motor_a = m.LargeMotor(motor_port_a)
motor_b = m.MediumMotor(motor_port_b)
#motor_c = m.LargeMotor(motor_port_c)
#motor_d = m.MediumMotor(motor_port_d)
#color_sensor = s.ColorSensor()
motor_a.reset()  # 모터 초기화
motor_b.reset()
#motor_c.reset()
#motor_d.reset()
SortMotor = motor_b
#SortMotor2 = motor_d
SortMotor.position=0
#SortMotor2.position=0

class WindowClass(QMainWindow, form_class):
    def __init__(self):
        super().__init__()
        self.setupUi(self)
        self.setWindowTitle("실시간 모니터링 시스템")
        
        self.menubar = self.findChild(QMenuBar,"menubar")
        filemenu = self.menubar.addMenu("결과 저장")
        
        저장=QAction('저장',self)
        저장.triggered.connect(self.save_table_to_excel)
        filemenu.addAction(저장)
        self.total = self.findChild(QLabel, "total")
        self.good = self.findChild(QLabel, "good")
        self.bad = self.findChild(QLabel, "bad")
        self.yellow = self.findChild(QLabel, "yellow")
        self.white = self.findChild(QLabel, "white")
       
        
        self.color_start= self.findChild(QPushButton,"color_start")
        self.color_start.clicked.connect(self.colorstart)
        
        self.color_stop= self.findChild(QPushButton,"color_stop")
        self.color_stop.clicked.connect(self.colorstop)
        
        self.defect_start= self.findChild(QPushButton,"defect_start")
        self.defect_start.clicked.connect(self.defectstart)
        
        self.defect_stop= self.findChild(QPushButton,"defect_stop")
        self.defect_stop.clicked.connect(self.defectstop)
        
        self.start = self.findChild(QPushButton,"start")
        self.start.clicked.connect(self.detectstart)
        
       
     
        self.tablewidget= self.findChild(QTableWidget,"tablewidget")
        
        self.tablewidget.setColumnCount(2)
        self.tablewidget.setEditTriggers(QAbstractItemView.NoEditTriggers)
        self.tablewidget.setHorizontalHeaderLabels(["불량_일련번호", "판정"])
            
 



    def save_table_to_excel(self):
    # 엑셀 저장 경로와 파일 이름 지정하기
        options = QFileDialog.Options()
        options |= QFileDialog.DontUseNativeDialog
        savepath, _ = QFileDialog.getSaveFileName(self, '엑셀 저장 경로', '', 'Excel Files (*.xlsx)', options=options)

        if savepath:
            # 파일 이름에 확장자 추가
            if not savepath.endswith('.xlsx'):
                savepath += '.xlsx'

            # 새로운 엑셀 워크북 생성
            workbook = Workbook()
            # 첫 번째 시트 선택
            sheet = workbook.active

            # TableWidget의 행과 열 개수 얻기
            rows = self.tablewidget.rowCount()
            cols = self.tablewidget.columnCount()

            # 테이블의 각 셀 값을 엑셀 시트에 저장
            for row in range(rows):
                for col in range(cols):
                    item = self.tablewidget.item(row, col)
                    if item is not None:
                        # 셀 값 가져오기
                        cell_value = item.text()

                        # 한글 데이터를 UTF-8로 인코딩
                        encoded_value = cell_value.encode('utf-8')

                        # 엑셀 시트에 셀 값 저장
                        sheet.cell(row=row + 1, column=col + 1, value=encoded_value.decode('utf-8'))

            # 엑셀 파일 저장
            workbook.save(filename=savepath)




  
    def colorstart(self):
        motor_c.run_forever(speed_sp=75)
    def colorstop(self):
        motor_c.stop()
    def defectstart(self):
        motor_a.run_forever(speed_sp=105)
    def defectstop(self):
        motor_a.stop()
        
    def detectstart(self):
        self.legoopt = argparse.Namespace(
            weights='before_best.pt',
            download=False,
            source='0',
            img_size=640,
            conf_thres=0.25,
            iou_thres=0.45,
            device='',
            view_img=False,
            save_txt=False,
            save_conf=False,
            nosave=False,
            classes=None,
            agnostic_nms=False,
            augment=False,
            update=False,
            project='runs/detect',
            name='object_tracking',
            exist_ok=False,
            no_trace=False,
            colored_trk=False,
            save_bbox_dim=False,
            save_with_object_id=False
        )
        self.timer = QTimer()
        self.timer.timeout.connect(self.update_table)
        self.timer.start(1000)
                                                    
        #self.timer2 = QTimer()
        #self.timer2.timeout.connect(self.update_label)
        #self.timer2.start(1000)                                            
                                                    
                                                    
        self.current_date = datetime.now().strftime("%Y%m%d")
        
        test_test.detect(self.legoopt, save_img=False)
        
      

    
                
    def update_table(self):
        classes = test_test.StoreClass.get_classes()
        self.tablewidget.setRowCount(len(classes))
        ok=0
        ng=0
        for i, class_name in enumerate(classes):
            item = QTableWidgetItem(f"{self.current_date}_{str(i+1)}")
            self.tablewidget.setItem(i, 0, item)
            
             # 두 번째 칼럼의 텍스트 읽기
            self.tablewidget.setItem(i, 1, QTableWidgetItem(class_name))
            class_name2 = self.tablewidget.item(i, 1).text() 
            if class_name2 == '양품':
                ok += 1
            elif class_name2 == '불량':
                ng += 1
                
        self.total.setText(str(self.tablewidget.rowCount()))
        self.good.setText(str(ok))
        self.bad.setText(str(ng))
                                                    
    def update_label(self):
        labeling_color = test_test.StoreColor.get_classes()
        whi=0
        yel=0
        
        for i, color_name in enumerate(labeling_color):
            if color_name == '흰색':
                whi += 1
            elif color_name =='노란색':
                yel += 1
        self.yellow.setText(str(whi))
        self.white.setText(str(yel))                                        
        
        
if __name__ == "__main__":
    app = QApplication(sys.argv)
    myWindow = WindowClass()
    
    myWindow.show()
    app.exec_()

In [2]:
pip install openpyxl

  Using cached openpyxl-3.1.2-py2.py3-none-any.whl (249 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [ ]:
python3 /usr/local/bin/rpyc_classic.py -m threaded --host 0.0.0.0